### Importing relevant packages

In [12]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

## Data

In [13]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

#Extracting training data and test data
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

#TFDataSets doesn't include validation data, so it is taken from the train dataset since it's the biggest

#Here's how it takes 10% of the training dataset
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples

#This 10% could not be a integer, but that wouldn´t make sense, so the variable is converted/casted into an int64 type variable
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

#Storing the number of test
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

### Creating a function to scale the inputs

In [14]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    #Since the pictures are in grayscale, they can only have 255 values, so dividing by thatr value, all of them will be between 0 and 1
    image/=255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

### Shuffling data

In [15]:
# A buffer is normally used with enormous datasets because the computer can´t handle  all the data at the same time

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)


#Creating a validation dataset from the shuffled data, with num_validation_samples data
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)

#Creating the train data with all the initial training data except for the validation data taken from it
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

# 1 < Batch_Size < #Samples
BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)

#Since validation data doesn´t use backpropagation, it's not required to batch it
#But, the model expect the validation dataset in batch form

validation_data = validation_data.batch(num_validation_samples)

#Test dataset doesn't needs to be batched
test_data = test_data.batch(num_test_samples)

validation_input, validation_targets = next(iter(validation_data))


## Model

### Outline the model

In [29]:
input_size = 784
output_size = 10
hidden_layer_size = 190

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
    tf.keras.layers.Dense(output_size, activation = 'softmax')
])

### Choosing the optimizer and the loss function

In [30]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training the model

In [31]:
NUM_EPOCHS = 5

model.fit(train_data, epochs = NUM_EPOCHS,
          validation_data=(validation_input, validation_targets),
          verbose = 2)




Epoch 1/5
540/540 - 4s - loss: 0.2761 - accuracy: 0.9193 - val_loss: 0.1243 - val_accuracy: 0.9632 - 4s/epoch - 7ms/step
Epoch 2/5
540/540 - 3s - loss: 0.1053 - accuracy: 0.9685 - val_loss: 0.0801 - val_accuracy: 0.9770 - 3s/epoch - 5ms/step
Epoch 3/5
540/540 - 3s - loss: 0.0713 - accuracy: 0.9777 - val_loss: 0.0621 - val_accuracy: 0.9813 - 3s/epoch - 6ms/step
Epoch 4/5
540/540 - 3s - loss: 0.0518 - accuracy: 0.9840 - val_loss: 0.0495 - val_accuracy: 0.9845 - 3s/epoch - 5ms/step
Epoch 5/5
540/540 - 3s - loss: 0.0411 - accuracy: 0.9875 - val_loss: 0.0378 - val_accuracy: 0.9893 - 3s/epoch - 6ms/step
